<h1> Parameter Tunning with MinMax
    
    GridCV for: learning rate 

# Importing libraries:

In [7]:
!pip install pandas
!pip install sklearn

import pandas as pd;
import numpy as np;
import sklearn
from sklearn.model_selection import train_test_split, cross_validate, cross_val_predict;
from sklearn.preprocessing import MinMaxScaler;
from sklearn import metrics;
from sklearn.model_selection import TimeSeriesSplit;

# Import data; set X and y; fill nan values and split in test and training  data:

In [8]:
# import data
data = pd.read_csv('Data_set_1_smaller.csv', index_col = 0)

# filter max values for offer if required
print(data.Offers.max()) #max is 2500... no need to filter max values

# 2017 & 2018 data
data = data.loc[data.index > 2018090000, :]

# reset index
data.reset_index(inplace = True)
data.drop('index', axis = 1, inplace = True)

# Divide features and labels
X = data.iloc[:, 0:15]
y = data.loc[:, 'Offers']

# Fill nan values (BEFORE OR AFTER TEST, TRAIN SPLIT!!!)
# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
X.fillna(X.mean(), inplace = True)
y.fillna(y.mean(), inplace = True)

X = X.astype('float64')
X = X.round(20)

# divide data into train and test with 20% test data
X_train, X_test, y_train, y_test = train_test_split(
         X, y, test_size=0.1, shuffle=False)

2500.0


C:\Users\maria\anaconda3\lib\site-packages\pandas\core\generic.py:6245: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


# Apply feature scaling:

In [9]:
sc_X = MinMaxScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)

# Importing the Keras libraries and packages:

In [10]:
import keras
from keras.models import Sequential # to initialise the NN
from keras.layers import Dense # to create layers
from keras.layers import Dropout
from keras import optimizers
from keras import initializers
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

# Create regressor:

In [11]:
# possible debug
X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

def regressor_tunning(n_hidden = 2, n_neurons = 30, kernel_initializer="he_normal",
    bias_initializer= initializers.Ones(), learning_rate = 0.001):
    model = Sequential()
    model.add(Dense(units = n_neurons, 
                    input_dim = 15))
    model.add(keras.layers.LeakyReLU(alpha = 0.2))
    for layer in range(n_hidden):
        model.add(Dense(n_neurons))
        model.add(keras.layers.LeakyReLU(alpha = 0.2))
    model.add(Dense(units = 1, 
                    activation = 'linear'))
    optimizer = optimizers.Adamax(lr = learning_rate)
    model.compile(loss = 'mse', optimizer = optimizer, metrics = ['mse', 'mae'])
    return model

# Apply Randomized tunning for the whole ANN:

In [12]:
regressor = KerasRegressor(build_fn = regressor_tunning)

# Dictionary to include the parameters
parameters = {'learning_rate': [0.00001, 0.0001, 0.001, 0.01, 0.1, 1]
               }

tscv = TimeSeriesSplit(n_splits = 6)

# add some early stopping
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='mse', patience = 15)

rnd_search_cv = GridSearchCV(estimator = regressor,
                                   param_grid = parameters,
                                   scoring = 'neg_mean_squared_error',
                                   n_jobs = -1,
                                   cv = tscv)

# checkpoints:
# early_stopping_cb = keras.callbacks.EarlyStopping(patience = 10, restore_best_weights = True)
# checkpoint_cb = keras.callbacks.ModelCheckpoint("my_keras_model.h5")

rnd_search_cv.fit(X_train, y_train, batch_size = 20, epochs = 100, callbacks=[early_stopping])


Epoch 1/100
264/264 [==============================] - 0s 1ms/step - loss: 11023.0312 - mse: 11023.0312 - mae: 89.5482
Epoch 2/100
264/264 [==============================] - 0s 1ms/step - loss: 2379.5315 - mse: 2379.5315 - mae: 26.6413
Epoch 3/100
264/264 [==============================] - 0s 1ms/step - loss: 2303.3396 - mse: 2303.3396 - mae: 27.0791
Epoch 4/100
264/264 [==============================] - 0s 1ms/step - loss: 2298.8733 - mse: 2298.8733 - mae: 26.9864
Epoch 5/100
264/264 [==============================] - 0s 1ms/step - loss: 2294.2085 - mse: 2294.2083 - mae: 26.9726
Epoch 6/100
264/264 [==============================] - 0s 1ms/step - loss: 2288.3083 - mse: 2288.3083 - mae: 26.8593
Epoch 7/100
264/264 [==============================] - 0s 1ms/step - loss: 2286.5601 - mse: 2286.5601 - mae: 26.9242
Epoch 8/100
264/264 [==============================] - 0s 1ms/step - loss: 2282.2507 - mse: 2282.2507 - mae: 26.9592
Epoch 9/100
264/264 [==============================] - 0s 1ms/

264/264 [==============================] - 0s 1ms/step - loss: 2193.2329 - mse: 2193.2329 - mae: 25.9389
Epoch 71/100
264/264 [==============================] - 0s 1ms/step - loss: 2194.8975 - mse: 2194.8975 - mae: 25.8358
Epoch 72/100
264/264 [==============================] - 0s 1ms/step - loss: 2192.1738 - mse: 2192.1741 - mae: 25.9073
Epoch 73/100
264/264 [==============================] - 0s 1ms/step - loss: 2191.2441 - mse: 2191.2439 - mae: 25.8524
Epoch 74/100
264/264 [==============================] - 0s 1ms/step - loss: 2192.2239 - mse: 2192.2239 - mae: 25.8929
Epoch 75/100
264/264 [==============================] - 0s 1ms/step - loss: 2190.5303 - mse: 2190.5303 - mae: 25.8832
Epoch 76/100
264/264 [==============================] - 0s 1ms/step - loss: 2188.9377 - mse: 2188.9380 - mae: 25.7379
Epoch 77/100
264/264 [==============================] - 0s 1ms/step - loss: 2188.1899 - mse: 2188.1899 - mae: 25.7195
Epoch 78/100
264/264 [==============================] - 0s 1ms/step -

GridSearchCV(cv=TimeSeriesSplit(max_train_size=None, n_splits=6),
             error_score=nan,
             estimator=<tensorflow.python.keras.wrappers.scikit_learn.KerasRegressor object at 0x000001EE7F7E8548>,
             iid='deprecated', n_jobs=-1,
             param_grid={'learning_rate': [1e-05, 0.0001, 0.001, 0.01, 0.1, 1]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='neg_mean_squared_error', verbose=0)

In [13]:
best_params_1 = rnd_search_cv.best_params_
best_score_1 = rnd_search_cv.best_score_
#results = rnd_search_cv.cv_results_

print("the best parameters are:{}".format(best_params_1))
print("the best score is:{}".format(best_score_1))
#print("results:".format(results))

the best parameters are:{'learning_rate': 0.001}
the best score is:-1946.111339014489
